[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

<!-- ![](https://i.imgur.com/qkg2E2D.png) -->

# AI Program

## Introduction to Estimation - The Maximum A Posteriori (MAP) Estimator

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 21/01/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0004EstimationMap.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Miscellaneous
from platform import python_version
import random
import warnings

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, interactive, IntSlider, Layout, VBox

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages


In [ ]:
# Parameters

numSamples = 1_000
σx = 1
µ0 = 2

## The Data

We'll generate data from the model:

$$ X \mid \mu \sim \mathcal{N} \left( \mu, \sigma_{x}^{2} \right), \; \mu \sim \mathcal{N} \left( {\mu}_{0}, {\sigma}_{\mu}^{2} \right) $$

Where the parameters ${\sigma}_{x}$, ${\mu}_{0}$ and ${\sigma}_{\mu}$ are known.

* <font color='brown'>(**#**)</font> In the model, instead of setting ${\sigma}_{\mu}^{2}$ explicitly, we'll use the ratio $\beta := \frac{ {\sigma}_{x}^{2} }{ {\sigma}_{\mu}^{2} }$.


In [ ]:
# Generate Data

vZ = (σx * np.random.randn(numSamples)) + µ0 #<! Generating many samples

* <font color='red'>(**?**)</font> Are the samples generated by the model? Pay attention to the mean of the generated data.

In [ ]:
# Plot of the Estimation Function

def PlotEstimation(vZ: np.ndarray, µ0: float = 0, ß: float = 1, numSamples: int = 100):
    
    # Validate input
    ß           = max(0.0001, ß)
    numSamples  = max(1, numSamples)
    
    vX      = vZ[:numSamples]
    meanX   = np.mean(vX)
    vRange  = [-10, 10]
    vMu     = np.linspace(vRange[0], vRange[1], 1001)
    vBins   = np.linspace(vRange[0], vRange[1], max(int(3 * np.sqrt(numSamples)), 5))
    vPdf    = sp.stats.norm.pdf(vMu, µ0, np.sqrt(1 / ß))
    vPdf   /= np.max(vPdf)
    
    µMAP = (µ0 * ß + numSamples * meanX) / (ß + numSamples)
    
    hF, hA = plt.subplots(figsize = (12, 5))
    hA.hist(vX, bins = vBins, facecolor = 'blue', alpha = 0.5, edgecolor = 'k', label = '$x_i$')
    hA.plot(vMu, vPdf * hA.get_ylim()[1], color = 'g', linewidth = 3, label = r'$f_{\mu}\left(\mu\right)$')
    hA.axvline(x = µMAP, linewidth = 3, color = 'r', linestyle = '-', label = r'$\hat{\mu}_{\mathrm{MAP}}$')
    hA.axvline(x = meanX, linewidth = 3, color = 'b', linestyle = '--', label = r'$\overline{x}$')
    hA.set_title(r'$\beta = ' + f'{ß:0.3f}' + '$')
    hA.legend()  


## The MAP Estimator

$$\boxed{\hat{\mu}_{\mathrm{MAP}}=\frac{ {\mu}_{0} \beta+N\overline{x}}{\beta+N}},\qquad\overline{x}=\frac{1}{N}\sum_{i=1}^{N}x_{i}, \qquad \beta:=\frac{\sigma_{x}^{2}}{\sigma_{\mu}^{2}}$$
$\beta\to0$: ignore prior  
$\beta\to\infty$: ignore observations 

In [ ]:
# Define the Plotting Function

hPlotEstimation = lambda µ0, ß, numSamples: PlotEstimation(vZ, µ0, ß, numSamples)

In [ ]:
# Interactive Visualization
µ0Slider = FloatSlider(min = -20, max = 20, step = 0.1, value = µ0, layout = Layout(width = '30%'))
ßSlider  = FloatSlider(min = 0, max = 250, step = 0.01, value = 0, layout = Layout(width = '30%'))
nSlider  = IntSlider(min = 1, max = numSamples, step = 1, value = 100, layout = Layout(width = '30%'))
interact(hPlotEstimation, µ0 = µ0Slider, ß = ßSlider, numSamples = nSlider, continuous_update = False)

plt.show()